In [ ]:
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as ss
import statsmodels.formula.api as smf
import matplotlib.collections as clt
import re
import pylab as pl

from nems_lbhb.pupil_behavior_scripts.mod_per_state import get_model_results_per_state_model
from nems_lbhb.pupil_behavior_scripts.mod_per_state import aud_vs_state
from nems_lbhb.pupil_behavior_scripts.mod_per_state import hlf_analysis
from nems_lbhb.stateplots import model_per_time_wrapper, beta_comp

%matplotlib inline
os.getcwd()

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 60)

In [ ]:
# AC SU+MU 307 DS' data (onBF and offBF together)
# run using beh_only_plot function in mod_per_state.py with state_list = ['st.fil0', 'st.fil'], 
# basemodel = '-ref-psthfr.s_stategain.S'
# loader = 'psth.fs20-ld-'
# fitter = '_jk.nf20-basic'
# get_model_results_per_state_model ---> returning df and saved on /auto/users/daniela/code/python_ACIC_proj/d_fil_307.csv 
# 12/18/18 DS
d_AC_307 = pd.read_csv('d_fil_307.csv')
d_AC_307 = d_AC_307.drop(['Unnamed: 0'], axis=1)
d_AC_307['experimenter'] = 'DS'
d_AC_307['area'] = 'AC'
d_AC_307['onBF'] = 'TBD'
d_AC_307['SU'] = False
d_AC_307.loc[d_AC_307['isolation']>=90.0, 'SU'] = True
d_AC_307['animal'] = d_AC_307['cellid'].map(lambda x: x[:3])
d_AC_307['task'] = 'TIN'

In [ ]:
d_AC_307.head()

In [ ]:
# AC 311 SVD's data (onBF only)
# run using beh_only_plot function in mod_per_state.py with state_list = ['st.fil0', 'st.fil'], 
# basemodel = '-ref-psthfr.s_stategain.S'
# loader = 'psth.fs20-ld-'
# fitter = '_jk.nf20-basic'
# get_model_results_per_state_model ---> returning df and saved on /auto/users/daniela/code/python_ACIC_proj/d_fil_311.csv 
# 12/18/18 DS
d_AC_311 = pd.read_csv('d_fil_311.csv')
d_AC_311 = d_AC_311.drop(['Unnamed: 0'], axis=1)
d_AC_311['onBF'] = True
d_AC_311['experimenter'] = 'SVD'
d_AC_311['area'] = 'AC'
d_AC_311['SU'] = False
d_AC_311['animal'] = d_AC_311['cellid'].map(lambda x: x[:3])
d_AC_311['task'] = 'TvN'
d_AC_311.loc[d_AC_311['isolation']>=90.0, 'SU'] = True

In [ ]:
d_AC_311.head()

In [ ]:
# IC SU 305 DS' data 
d_IC_305 = pd.read_csv('d_305_IC_new.csv') # run with stategain model on October 31st
d_IC_305 = d_IC_305.drop(['Unnamed: 0'], axis=1)
d_IC_305['onBF'] = 'TBD'
d_IC_305['experimenter'] = 'DS'
d_IC_305['area'] = 'IC'
d_IC_305['animal'] = d_IC_305['cellid'].map(lambda x: x[:3])
d_IC_305['task'] = 'TIN'
d_IC_305.loc[d_IC_305['animal']=='ley', 'task'] = 'TvN'
d_IC_305['SU'] = True

In [ ]:
d_IC_305.head()

In [ ]:
# IC SU 295 Sean's data 
d_IC_295 = pd.read_csv('d_295_IC_old.csv')
d_IC_295 = d_IC_295.drop(['Unnamed: 0'], axis=1)
d_IC_295['onBF'] = 'TBD'
d_IC_295['experimenter'] = 'SJS'
d_IC_295['area'] = 'IC'
d_IC_295['task'] = 'TvN'
d_IC_295['SU'] = True

In [ ]:
d_IC_295.head()

In [ ]:
# concat all dataframes
d_all = pd.concat([d_AC_311, d_AC_307, d_IC_295, d_IC_305], sort=False)

In [ ]:
d_all.head()

In [ ]:
#d_all[d_all['cellid']=='BRT016f-a1']

In [ ]:
# generated by running get_ptd_tuning in MATLAB and then running struct2csv(cellframe, 'tuning_info_batch_295.csv')
# IC data Sean's SU only
d_295_strf = pd.read_csv('tuning_info_batch_295.csv')
d_295_strf = d_295_strf.drop(['Unnamed: 24'], axis=1)

In [ ]:
# generated by running get_ptd_tuning in MATLAB and then running struct2csv(cellframe, 'tuning_info_batch_307.csv')
# AC data DS' SU+MU
d_307_strf = pd.read_csv('tuning_info_batch_307.csv')
d_307_strf = d_307_strf.drop(['Unnamed: 43'], axis=1)

In [ ]:
# generated by running get_ptd_tuning in MATLAB and then running struct2csv(cellframe, 'tuning_info_batch_309.csv')
# IC data DS' SU+MU (305 is SU only)
d_309_strf = pd.read_csv('tuning_info_batch_309.csv')
d_309_strf = d_309_strf.drop(['Unnamed: 43'], axis=1)

In [ ]:
# concat all strf dataframes
d_strf = pd.concat([d_295_strf, d_307_strf, d_309_strf], sort=False)

In [ ]:
d_strf.head()

In [ ]:
# merge all into one giant df
df = pd.merge(left=d_all, right=d_strf, how='outer', on='cellid')

In [ ]:
df.head()

In [ ]:
# len(d_295_strf[d_295_strf['ACTIVE_0'].isna()])
# 64 NAN for ACTIVE_0
# 10 not nan for ACTIVE_0  (No pre-passive)

In [ ]:
df.columns

In [ ]:
def fix_TBD_onBF(df):
    '''This function takes the monster dataframe and adds True to the column 'onBF' when 'ACTIVE_1_tardist' is
    within 0.5 --> half an octave between unit BF and target frequency in ACTIVE 1'''
    BF_TBD = (df['onBF']=='TBD')
    df.loc[BF_TBD, 'onBF'] = df.loc[BF_TBD, 'ACTIVE_1_tardist'].map(lambda x: abs(x)<=0.5) #WOW!
    return df

In [ ]:
df['onBF'].value_counts()

In [ ]:
df = fix_TBD_onBF(df)

In [ ]:
df['onBF'].value_counts()

In [ ]:
def plotme(df, x_col, y_col, title=None):
    ax = sns.violinplot(x=x_col, y=y_col, data=df)
    #ax = sns.barplot(x=x_col, y=y_col, data=df)
    #ax = sns.swarmplot(x=x_col, y=y_col, data=df, color='black')
    plt.axhline(color='k', linestyle='--', linewidth=0.5)
    plt.ylabel(y_col)
    if title:
        plt.title(title)
        plt.savefig(title+x_col+y_col+'.pdf')

In [ ]:
# variables to slice up df 
AC = df['area']=='AC'
IC = df['area']=='IC'
onBF = df['onBF']==True
offBF = df['onBF']==False
active1 = df['state_chan']=='ACTIVE_1'

In [ ]:
plotme(df[AC & active1], 'experimenter', 'g', title='AC_P0A1')

In [ ]:
plotme(df[AC & onBF & active1], 'experimenter', 'MI', title='AC_onBF_P0A1')

In [ ]:
plotme(df[AC & onBF & active1], 'experimenter', 'g', title='AC_onBF_P0A1')

In [ ]:
plotme(df[IC & onBF & active1], 'task', 'g', title='IC_onBF_P0A1')

In [ ]:
plotme(df[IC & offBF & active1], 'task', 'g', title='IC_offBF_P0A1')